# Generate financial time series (FTS) data

In [1]:
import re
import sys
import warnings
import pandas as pd # type: ignore
import yfinance as yf # type: ignore

sys.path.append('../modules')
import get_financial_time_series as get_fts # type: ignore

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

## Global variables

In [2]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
output_path = "../output_files"
input_generation_date = "2024-06-24"

In [3]:
# Standard and Poor's 500 - S&P 500 (United States)
# Source https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
# "2000-01-01" - "2022-12-31"
# GEV: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
# KVUE: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
# SOLV: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
# VLTO: Data doesn't exist for startDate = 946702800, endDate = 1672462800")

df_stock_index = pd.read_csv("{}/sp500_tickers.csv".format(input_path_data_dictionary), low_memory = False)

# Mexico Price and Quote Index (Mexico)
# source https://es.finance.yahoo.com/quote/%5EMXX/components/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAICc-QDC-sGwCChRyZU_MzY5IZIpLgF6EOVbbq-P7L_F6E0ur0Zz-MAwE1zXvNewk1qXCrCh4tU6A48XLFTPTaxgwdtcHisQnftQhKrJjlhLFjzRswWW8ybr08epnyjiWfra8DQjOdPTIIBrNB86THdyO7MnRmHpizHkFyNPL1Np
# "2005-01-03" - "2022-12-31"

#df_stock_index = pd.read_csv("{}/ipc_mexico_tickers.csv".format(input_path_data_dictionary), low_memory = False)
list_stock_indexes = df_stock_index.set_index("ticker")["ticker_name"].to_dict()

## Download time series - Stock indexes

In [4]:
df_market = get_fts.process_financial_time_series(
    ticker_dict = list_stock_indexes,
    initial_date = "2000-01-01",
    final_date = "2024-12-31",#input_generation_date,
    interval = "1d"
)

----------------------------------- YAHOO FINANCE DATA -----------------------------------

- Download A with initial 6162 rows and 6115 rows after profiling
- Processed A : AGILENT TECHNOLOGIES
- Download AAL with initial 4721 rows and 4680 rows after profiling
- Processed AAL : AMERICAN AIRLINES GROUP
- Download AAPL with initial 6162 rows and 6130 rows after profiling
- Processed AAPL : APPLE INC.
- Download ABBV with initial 2893 rows and 2876 rows after profiling
- Processed ABBV : ABBVIE
- Download ABNB with initial 893 rows and 890 rows after profiling
- Processed ABNB : AIRBNB
- Download ABT with initial 6162 rows and 6099 rows after profiling
- Processed ABT : ABBOTT
- Download ACGL with initial 6162 rows and 6007 rows after profiling
- Processed ACGL : ARCH CAPITAL GROUP
- Download ACN with initial 5773 rows and 5725 rows after profiling
- Processed ACN : ACCENTURE
- Download ADBE with initial 6162 rows and 6135 rows after profiling
- Processed ADBE : ADOBE INC.
- Download AD

## Save data in input files for no reprocessing

In [5]:
# Standard and Poor's 500 - S&P 500 (United States)
df_market.to_pickle("{}/df_stock_index_gspc_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))

# Mexico Price and Quote Index (Mexico)
#df_market.to_pickle("{}/df_stock_index_mxx_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))
df_market

,date,symbol,Adj Close,return,log_return,z_score_log_return
0,2000-01-04,A,40.281460,-3.331558,0.000000,0.000000
1,2000-01-05,A,37.782803,-2.498657,-0.064037,-2.494965
2,2000-01-06,A,36.344177,-1.438625,-0.038820,-1.514934
3,2000-01-07,A,39.372852,3.028675,0.080043,3.104458
4,2000-01-10,A,41.757954,2.385101,0.058813,2.279404
...,...,...,...,...,...,...
2711967,2024-06-25,ZTS,167.979996,-3.860001,-0.022719,-1.441045
2711968,2024-06-26,ZTS,170.899994,2.919998,0.017234,1.027639
2711969,2024-06-27,ZTS,176.460007,5.560013,0.032016,1.941028
2711970,2024-06-28,ZTS,173.360001,-3.100006,-0.017724,-1.132402
